In [1]:
import cv2

In [2]:
import pandas as pd
from pathlib import Path
import numpy as np
import os
from sklearn.model_selection import train_test_split
from keras_preprocessing.image import ImageDataGenerator

from keras import applications
from keras.models import Sequential,Model
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPool2D, BatchNormalization,Input,MaxPooling2D,GlobalMaxPooling2D,concatenate
from keras.layers import GlobalAveragePooling2D
from keras.applications import ResNet50
from keras.callbacks import ModelCheckpoint
import tensorflow as tf


from efficientnet.keras import EfficientNetB3
from keras.applications.vgg16 import VGG16
import keras

import logging

import keras.backend as K
from keras_radam import RAdam

from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint

import matplotlib.pyplot as plt



2023-09-21 10:57:47.325580: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
directory = Path("./ODIR-5K_Training_Dataset")

i = 0
for item in os.listdir(directory):
    i += 1

print(i)

7000


In [4]:
directory = Path("./ODIR-5K_Testing_Images")

i = 0
for item in os.listdir(directory):
    i += 1

print(i)

1000


In [5]:
df = pd.DataFrame(pd.read_excel("ODIR-5K_Training_Annotations(Updated)_V2.xlsx"))

In [6]:
df

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1
3,3,66,Male,3_left.jpg,3_right.jpg,normal fundus,branch retinal artery occlusion,0,0,0,0,0,0,0,1
4,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,4686,63,Male,4686_left.jpg,4686_right.jpg,severe nonproliferative retinopathy,proliferative diabetic retinopathy,0,1,0,0,0,0,0,0
3496,4688,42,Male,4688_left.jpg,4688_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0
3497,4689,54,Male,4689_left.jpg,4689_right.jpg,mild nonproliferative retinopathy,normal fundus,0,1,0,0,0,0,0,0
3498,4690,57,Male,4690_left.jpg,4690_right.jpg,mild nonproliferative retinopathy,mild nonproliferative retinopathy,0,1,0,0,0,0,0,0


In [7]:
train, validation = train_test_split(df, test_size=(500/3500), random_state=42, shuffle=True)

In [8]:
# determine the path where to save the train and test file
data_dir = Path('./datasplits')
train_path = Path(data_dir, 'train.csv')
validation_path = Path(data_dir, 'validation.csv')

# save the train and test file
# again using the '\t' separator to create tab-separated-values files
train.to_csv(train_path, index=False)
validation.to_csv(validation_path, index=False)

In [9]:
train.head()

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O
1211,1291,63,Male,1291_left.jpg,1291_right.jpg,glaucoma，macular epiretinal membrane,glaucoma，macular epiretinal membrane,0,0,1,0,0,0,0,1
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0
1497,1868,63,Male,1868_left.jpg,1868_right.jpg,normal fundus,dry age-related macular degeneration,0,0,0,0,1,0,0,0
1444,1644,63,Female,1644_left.jpg,1644_right.jpg,refractive media opacity,pathological myopia,0,0,0,0,0,0,1,1
3366,4535,59,Male,4535_left.jpg,4535_right.jpg,normal fundus,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0


In [10]:
validation.head()

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O
1650,2192,50,Female,2192_left.jpg,2192_right.jpg,cataract,cataract,0,0,0,1,0,0,0,0
2456,3079,66,Female,3079_left.jpg,3079_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0
2232,2855,45,Female,2855_left.jpg,2855_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0
1945,2568,65,Female,2568_left.jpg,2568_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0
309,310,33,Male,310_left.jpg,310_right.jpg,normal fundus,macular epiretinal membrane,0,0,0,0,0,0,0,1


In [12]:
batch_size = 15
img_shape = 224

In [13]:
def get_flow_from_dataframe(generator, dataframe,directory_path,
                            image_shape=(img_shape, img_shape),
                            #subset='training',
                             batch_size=35):
    train_generator_1 = generator.flow_from_dataframe(dataframe,
                                                      directory=directory_path,
                                                      target_size=(img_shape,img_shape),
                                                      x_col="Left-Fundus",
                                                      y_col=['N'	,'D'	,'G',	'C',	'A'	,'H'	,'M',	'O'],
                                                      class_mode='other',
                                                      shuffle=False,
                                                      batch_size=batch_size,
                                                      seed=7,
                                                      classes=['N'	,'D'	,'G',	'C',	'A'	,'H'	,'M',	'O']
                                                      #subset=subset
                                                      )

    train_generator_2 = generator.flow_from_dataframe(dataframe, 
                                                      directory=directory_path,
                                                      target_size=(img_shape,img_shape),
                                                      x_col="Right-Fundus",
                                                      y_col=['N'	,'D'	,'G',	'C',	'A'	,'H'	,'M',	'O'],
                                                      class_mode='other',
                                                      shuffle=False,
                                                      batch_size=batch_size,
                                                      seed=7,
                                                      classes=['N'	,'D'	,'G',	'C',	'A'	,'H'	,'M',	'O']                                                     
                                                      #subset=subset
                                                      )
    while True:
        x_1 = train_generator_1.next()
        x_2 = train_generator_2.next()

        yield [x_1[0], x_2[0]], x_1[1]
     

In [16]:

def load_ben_color(image, sigmaX=10):
    #image = cv2.imread(path)
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (img_shape, img_shape))
    image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
    
    
    return image

In [17]:

from keras_preprocessing.image import ImageDataGenerator

train_data_gen= ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=.1,
    rescale=1/255,
    fill_mode='nearest',
    shear_range=0.1,
    brightness_range=[0.5, 1.5],
    preprocessing_function=load_ben_color
)

val_data_gen= ImageDataGenerator(
    rescale=1/255
    
)

In [14]:

train_gen = get_flow_from_dataframe(train_data_gen, train, image_shape=(img_shape, img_shape),
                                        directory_path="data/ODIR-5K_Training_Dataset/",batch_size=batch_size)
valid_gen = get_flow_from_dataframe(train_data_gen, validation, image_shape=(img_shape, img_shape),
                                        directory_path="data/ODIR-5K_Training_Dataset/",batch_size=batch_size)
test_gen = get_flow_from_dataframe(train_data_gen, test, image_shape=(img_shape, img_shape),
                                        directory_path="data/ODIR-5K_Testing_Images/",batch_size=batch_size)

NameError: name 'train_data_gen' is not defined

In [17]:
for i in ['N'	,'D'	,'G',	'C',	'A'	,'H'	,'M',	'O']:
  print(validation[i].value_counts())

0    353
1    147
Name: N, dtype: int64
0    341
1    159
Name: D, dtype: int64
0    475
1     25
Name: G, dtype: int64
0    469
1     31
Name: C, dtype: int64
0    476
1     24
Name: A, dtype: int64
0    476
1     24
Name: H, dtype: int64
0    478
1     22
Name: M, dtype: int64
0    336
1    164
Name: O, dtype: int64


In [18]:
test = pd.read_csv('XYZ_ODIR.csv')

In [19]:
test

,ID,N,D,G,C,A,H,M,O
0,937,0,0,0,0,0,0,0,0
1,967,0,0,0,0,0,0,0,0
2,988,0,0,0,0,0,0,0,0
3,995,0,0,0,0,0,0,0,0
4,1000,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
495,4787,0,0,0,0,0,0,0,0
496,4788,0,0,0,0,0,0,0,0
497,4789,0,0,0,0,0,0,0,0
498,4791,0,0,0,0,0,0,0,0


In [20]:
left_test=[]
right_test=[]
ID=[]
for i in test['ID']:
  ID.append(i)
  left_test.append(str(i)+'_left.jpg')
  right_test.append(str(i)+'_right.jpg')

name_df=pd.DataFrame(zip(ID,left_test,right_test),columns=['ID','Left-Fundus','Right-Fundus'])
name_df.head()
result = pd.merge(name_df, test, on='ID')
result.head()

,ID,Left-Fundus,Right-Fundus,N,D,G,C,A,H,M,O
0,937,937_left.jpg,937_right.jpg,0,0,0,0,0,0,0,0
1,967,967_left.jpg,967_right.jpg,0,0,0,0,0,0,0,0
2,988,988_left.jpg,988_right.jpg,0,0,0,0,0,0,0,0
3,995,995_left.jpg,995_right.jpg,0,0,0,0,0,0,0,0
4,1000,1000_left.jpg,1000_right.jpg,0,0,0,0,0,0,0,0


In [21]:
img_shape=300

In [22]:

#https://www.kaggle.com/ratthachat/aptos-eye-preprocessing-in-diabetic-retinopathy
def crop_image1(img,tol=7):
    # img is image data
    # tol  is tolerance
        
    mask = img>tol
    return img[np.ix_(mask.any(1),mask.any(0))]

def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img

In [23]:

def load_ben_color(image, sigmaX=10):
    #image = cv2.imread(path)
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (img_shape, img_shape))
    image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
    
    
    return image

In [24]:
train_data_gen= ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=.1,
    rescale=1/255,
    fill_mode='nearest',
    shear_range=0.1,
    brightness_range=[0.5, 1.5],
    preprocessing_function=load_ben_color
)

val_data_gen= ImageDataGenerator(
    rescale=1/255
    
)

In [25]:
def get_flow_from_dataframe(generator, dataframe,directory_path,
                            image_shape=(img_shape, img_shape),
                            #subset='training',
                             batch_size=35):
    train_generator_1 = generator.flow_from_dataframe(dataframe,
                                                      directory=directory_path,
                                                      target_size=(img_shape,img_shape),
                                                      x_col="Left-Fundus",
                                                      y_col=['N'	,'D'	,'G',	'C',	'A'	,'H'	,'M',	'O'],
                                                      class_mode='other',
                                                      shuffle=False,
                                                      batch_size=batch_size,
                                                      seed=7,
                                                      classes=['N'	,'D'	,'G',	'C',	'A'	,'H'	,'M',	'O']
                                                      #subset=subset
                                                      )

    train_generator_2 = generator.flow_from_dataframe(dataframe, 
                                                      directory=directory_path,
                                                      target_size=(img_shape,img_shape),
                                                      x_col="Right-Fundus",
                                                      y_col=['N'	,'D'	,'G',	'C',	'A'	,'H'	,'M',	'O'],
                                                      class_mode='other',
                                                      shuffle=False,
                                                      batch_size=batch_size,
                                                      seed=7,
                                                      classes=['N'	,'D'	,'G',	'C',	'A'	,'H'	,'M',	'O']                                                     
                                                      #subset=subset
                                                      )
    while True:
        x_1 = train_generator_1.next()
        x_2 = train_generator_2.next()

        yield [x_1[0], x_2[0]], x_1[1]

In [26]:
batch_size=15
train_gen = get_flow_from_dataframe(train_data_gen, train, image_shape=(img_shape, img_shape),
                                        directory_path="./ODIR-5K_Training_Dataset/",batch_size=batch_size)
valid_gen = get_flow_from_dataframe(train_data_gen, validation, image_shape=(img_shape, img_shape),
                                        directory_path="./ODIR-5K_Training_Dataset/",batch_size=batch_size)
test_gen = get_flow_from_dataframe(train_data_gen, result, image_shape=(img_shape, img_shape),
                                        directory_path="./ODIR-5K_Testing_Images/",batch_size=batch_size)
     

In [27]:
img_shape

300

In [63]:
tf.get_logger().setLevel(logging.ERROR)


In [39]:
# from tensorflow.keras.layers import Input
# from tensorflow.keras.applications import EfficientNetB3
# import ssl

# # Create symbolic input tensors
# left_input = Input(shape=(img_shape, img_shape, 3))
# right_input = Input(shape=(img_shape, img_shape, 3))

# # Create EfficientNetB3 models using the symbolic input tensors
# left_model = EfficientNetB3(
#     input_shape=(img_shape, img_shape, 3),
#     include_top=False,
#     weights='imagenet',
#     input_tensor=left_input
# )
# right_model = EfficientNetB3(
#     input_shape=(img_shape, img_shape, 3),
#     include_top=False,
#     weights='imagenet',
#     input_tensor=right_input
# )

# # Set models as non-trainable
# left_model.trainable = False
# right_model.trainable = False


43941136/43941136 [==============================] - 11s 0us/step


In [64]:
from tensorflow.keras.applications import EfficientNetB3
from keras.applications.vgg16 import VGG16
import keras
left_input=Input(shape=(img_shape,img_shape,3))
right_input=Input(shape=(img_shape,img_shape,3))

left_model = EfficientNetB3(input_shape =  (img_shape,img_shape,3), include_top = False, weights = 'imagenet',input_tensor=left_input)
right_model = EfficientNetB3(input_shape =  (img_shape,img_shape,3), include_top = False, weights = 'imagenet',input_tensor=right_input)

left_model.trainable = False
right_model.trainable = False

In [65]:
con = concatenate([left_model.output, right_model.output])
gap_dr = Dropout(0.5)(con)
GAP= GlobalAveragePooling2D()(gap_dr)
fc1=Dense(256, activation = 'relu')(GAP)
fc2=Dense(128, activation = 'relu')(fc1)
fc3=Dense(64, activation = 'relu')(fc2)
out = Dense(8, activation = 'sigmoid')(fc3)

In [69]:
def rename_layers(model, suffix):
    for layer in model.layers:
        layer._name = layer.name + suffix  # Use _name to set the layer's name

# Rename the layers
rename_layers(left_model, '_left')
rename_layers(right_model, '_right')



In [70]:
model =Model(inputs=[left_input, right_input], outputs=out)

In [71]:
def focal_loss(y_true, y_pred, alpha = 0.75, gamma = 2):
  alpha_factor = K.ones_like(y_true) * alpha
  alpha_factor = tf.where(K.equal(y_true, 1), alpha_factor, 1 - alpha_factor)
  focal_weight = tf.where(K.equal(y_true, 1), 1 - y_pred, y_pred)
  focal_weight = alpha_factor * focal_weight ** gamma
  loss = focal_weight * K.binary_crossentropy(y_true, y_pred)
  return loss

In [72]:
model.compile('adam',loss=focal_loss,metrics=['accuracy'])

In [73]:
step_size_train=len(train)/batch_size
step_size_val=len(validation)/batch_size


In [74]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3,
                              patience=3, min_lr=0.000001, verbose=1)
checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [78]:
train_gen.reset()
valid_gen.reset()

AttributeError: 'generator' object has no attribute 'reset'

In [80]:

hist=model.fit(train_gen, epochs=20,
                        steps_per_epoch=step_size_train,
                        use_multiprocessing=True,
                        validation_data=valid_gen,
                        validation_steps=step_size_val,
                        verbose=2,callbacks=[reduce_lr,checkpoint],
                    
                        )
     

StopIteration: 

In [67]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

NameError: name 'hist' is not defined

In [ ]:

y_pred = model.predict_generator(valid_gen,steps=step_size_val)
y_pred1=y_pred.copy()
y_pred1[y_pred1>=0.5] = 1
y_pred1[y_pred1<0.5] = 0
y_pred1=y_pred1.astype(int)

In [ ]:
y_test= model.predict_generator(test_gen,steps=len(test)/batch_size)


In [ ]:
for i,j in enumerate(['N'	,'D'	,'G',	'C',	'A'	,'H'	,'M',	'O']):
  test[j]=y_test[:,i]

In [ ]:
test.head()